# Лабораторная работа №6. Машины опорных векторов и ядра скалярного произведения

В данной лабораторной работе рассматривается использование машин опорных векторов для решения задач классификации и регрессии.

In [ ]:
import numpy as np
import sklearn as sk
import sklearn.datasets
import sklearn.model_selection
import sklearn.svm 
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['axes.grid'] = True
plt.rcParams['figure.constrained_layout.use'] = True

## Задание №1

Подготовьте обучающую и тестовую выборки.
Значение `random_state` соответствует номеру варианта.
Размер тестовой выборки должен составлять 20% от исходной выборки.

In [ ]:
x, y = sk.datasets.make_classification(n_samples=1000, n_features=2, n_informative=2, n_redundant=0, n_clusters_per_class=2, random_state=???)
x_train, x_test, y_train, y_test = ??? random_state=42

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(2*5, 4))
ax0.scatter(???)
ax0.set(xlabel='x0', ylabel='x1', title='Обучающая выборка')
ax1???

print(x_train.shape, y_train.shape)

## Задание №2

Рассмотрим задачу классификации с использованием машин опорных векторов.
Визуализируйте разделение пространства признаков для следующих ядер:

* линейное,
* полиномиальные (порядков 2-5),
* радиальное RBF.

Для каждого из ядер найдите точность (accuracy) для обучающей и тестовой выборок, а также число используемых опорных векторов (сложность модели).
На основе полученных значений точности (accuracy) выберите наилучшую модель.
Для полиномиальных ядер в отчет включите только ту модель, которая дает наилучший результат.
Число опорных векторов - это сумма опорных векторов всех классов в переменной `n_support_`.

In [ ]:
x_min, x_max = x_test[:, 0].min() - 1, x_test[:, 0].max() + 1
y_min, y_max = x_test[:, 1].min() - 1, x_test[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05), np.arange(y_min, y_max, 0.05))

models = {'linear': sk.svm.SVC(kernel=???), ???}
for p in range(2, 6):
#for p in [???]:
    models[f'poly{p}'] = ???

_, axes = plt.subplots(1, len(models), figsize=(len(models)*5, 4))
for (name, model), ax in zip(models.items(), axes):
    model???
    Z = model.predict(np.c_[xx.ravel(), yy.ravel(), np.zeros((xx.ravel().shape[0], x_test.shape[1] - 2))])
    ax.contourf(xx, yy, Z.reshape(xx.shape), cmap=plt.cm.Paired, alpha=0.5)
    ax.scatter(???)
    ax.set(xlabel='x0', ylabel='x1', title=name)
    
    print(name, 'train score:', model.score(x_train, y_train), 'test score:', model.score(x_test, y_test), 'n_support:', ???)

**Вопросы:**

1. Какая модель оказалась лучшей по точности классификации?
1. Какая модель использует наименьшее число опорных векторов?
1. Что можно сказать о влиянии числа опорных векторов на качество модели?

**Ответы:**

1. ...
1. ...

## Задание №3

Пусть задача минимизации функции стоимости имеет вид
$$
\frac{1}{2} \lVert\mathbf{w}\rVert^2 + C \sum_{i=0}^{N-1} \xi_i \rightarrow \min_{w,\xi},
$$
а преобразование пространства признаков осуществляется ядром
$$
RBF(x, y) = \exp\left( -\frac{\lVert x - y \rVert^2}{\sigma^2} \right),
$$
где $C > 0$, $\sigma > 0$.

**Замечания:**

* Оптимальные параметры следует выбирать по наилучшему значению на тестовой выборке.
Если в исследуемой зависимости наблюдается несколько пиков с наилучшим значением, то стоит принять в рассмотрение также результат на обучающей выборке.
* Параметру $\sigma$ в `SVC` соответствует параметр `gamma`.
* Выбирайте диапазоны параметроы $C$ и $\sigma$ такими, чтобы было хорошо видно поведение зависимостей на всех графиках.
* Для генерации параметров $C$ используйте `np.linspace`, для $\sigma$ - `np.logspace`.
* Ширина зазора - это квадратичная норма от результата `desision_function` (помните, что в `numpy` функция `np.linalg.norm` выдаёт ненормированную норму).

### Задание №3.1

* Постройте графики зависимостей числа опорных векторов и ширины зазора от параметра регуляризации $C$, приняв $\sigma$ равным единице.
* Постройте графики зависимостей точности (accuracy) от параметра $C$ для обучающей и тестовой выборок, на основе чего выберите оптимальный параметр $C$.

In [ ]:
%%time

param_C = np.linspace(???)
metrics = {'distance': [], 'count': [], 'train': [], 'test': []}
for C in param_C:
    model = sk.svm.SVC(???, decision_function_shape='ovo')
    ???
    metrics['distance'].append(???)
    ???

_, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(3*5, 4))
???

C_opt = param_C[???] np,flatnonzero, np.amax, np.argmax
print(f"Optimal C = {C_opt}, test accuracy = {???}, train accuracy = {???}")

**Вопросы:**

1. Какое значение принимает оптимальный параметр регуляризации `C`?
1. Как ведёт себя ширина зазора при увеличении параметра `C`?
1. Что происходит с количеством опорных векторов при увеличении параметра `C`?

**Ответы:**

1. ...
1. ...

### Задание №3.2

* Постройте графики зависимостей числа опорных векторов и ширины зазора от параметра $\sigma$, используя найденное оптимальное $C$.
* Постройте графики зависимостей точности (accuracy) от параметра $\sigma$ для обучающей и тестовой выборок, на основе чего выберите оптимальный параметр $\sigma$.
* Для параметра $\sigma$ используйте логарифмический масштаб.

In [ ]:
%%time

param_sigma = np.logspace(???)
metrics = {'distance': [], 'count': [], 'train': [], 'test': []}
for sigma in param_sigma:
    model = ???
    metrics['distance']???

???
ax0.semilogx(???)
ax0.set(xlabel='$\log_{10}(\sigma)$', ylabel='Расстояние', title='Ширина зазора')
???
ax2.legend()

sigma_opt = param_sigma[???]
print(f"Optimal sigma = {sigma_opt}, test accuracy = {???}, train accuracy = {???}")

**Вопросы:**

1. Как ведёт себя значение ширины зазора при изменении значения параметра $\sigma$?
1. Как изменяется число опорных векторов при уменьшении/увеличении значения парметра $\sigma$?
1. Удалось ли улучшить результаты классификации?

**Ответы:**

1. ...
1. ...

### Задание №3.3

Визуализируйте разделение пространства признаков для наилучшей модели.

In [ ]:
model = sk.svm.SVC(kernel='rbf', C=C_opt, gamma=sigma_opt)
???
Z = model.predict(np.c_[xx.ravel(), yy.ravel(), np.zeros((xx.ravel().shape[0], x_test.shape[1] - 2))])
plt.contourf(xx, yy, Z.reshape(xx.shape), cmap=plt.cm.Paired, alpha=0.5)
plt.scatter(???)
plt.xlabel('x0')
plt.ylabel('x1')

**Вопросы:**

* Удалось ли увеличить качество модели (со скольких до скольких) по сравнению с предыдущим заданием?
Как при этом изменилась форма разделяющей поверхности? 
* Сделайте вывод о влиянии параметров $C$ и $\sigma$ на сложность модели и ширину зазора.

**Ответы:**

1. ...
1. ...

## Задание №4

Подготовьте датасет с нелинейными данными. Координата `x` генериурется как отрезок $[-1; 1]$ в количестве 100 отсчётов, `y` генерируется согласно варианту.
Разделите датасет на обучающую и тестовую выборки.
Размер тестовой выборки следует выбрать равным 20% от размера всей выборки.

Функция для генерации `y` компоненты датасета (выбирается согласно формуле `1 + (<порядковый номер> % 2)`:

1. `np.arctan(A * x - 0.23) + 0.1`
1. `np.sin(A * x + 0.1) - 0.43`

Число `A` вычисляется по формуле `4 + (<порядковый номер> % 5)`.

In [ ]:
x = np.linspace(???)
y = ???
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(np.expand_dims(x, -1), y, test_size=???, random_state=78)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(2*6, 4))
ax0.plot(x_train, y_train, 'o')
ax0.set(xlabel='Номер отсчёта', ylabel='y', title='Обучающая выборка')
???
print(x_train.shape, y_train.shape)

## Задание №5

Рассмотрим задачу регрессии с использованием машин опорных векторов (`sk.svm.SVR`).
Постройте графики, на которых изображены истинные и предсказанные значения для следующих ядер:

* линейное,
* полиномиальные (порядков 2-10),
* радиальное RBF.

Для каждого из ядер найдите MSE-ошибки на обучающей и тестовой выборках.
На основе полученных значений MSE выберите наилучшую модель.
Для полиномиальных ядер в отчет включите только ту модель, которая дает наилучший результат.

In [ ]:
models = {'linear': sk.svm.SVR(kernel=???), ???}
for i in range(2, 11):
#for i in [???]:
    models[f'poly{i}'] = ???

fig, axes = plt.subplots(1, len(models), figsize=(len(models)*5, 4))
for (name, model), ax in zip(models.items(), axes):
    ???
    ax.plot(x_test, y_test, 'o', label='true')
    ax.plot(???, 's', label='predicted')
    ax.set(xlabel='Номер отсчёта', ylabel='y', title=name)
    ax.legend()
    
    err_train = ???
    err_test = ???
    print(name, 'train MSE:', err_train, 'test MES:', err_test, 'n_support:', ???)

**Вопросы:**

1. Насколько модели показывают правдоподобные результаты?
1. Какая модель даёт меньшую ошибку MSE?
1. Какая модель использует меньшее количество опорных векторов?

**Ответы:**

1. ...
1. ...

## Задание №6

Попробуйте улучшить результат, полученный оптимальной моделью из предыдущего задания.
Для этого следует подобрать параметры регуляризации $C$ и $\varepsilon$ одновременно с помощью функции
[GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
При построении решётки параметров для `C` используйте `np.linspace`, для `epsilon` - `np.logspace`.
Можно подбирать диапазоны поэтапно: от широкого, но с большим шагом, до более маленьго диапазона с меньшим шагом.
Найти самый оптимальный минимум не требуется.
Вполне можно остановиться на MSE $\approx 10^{-5}$.

### Задание №6.1

Постройте график, на котором изображены истинные и предсказанные значения для наилучшей модели.

In [ ]:
%%time

param_grid = {'C': np.linspace(???), 'epsilon': np.logspace(???)}
model = sk.model_selection.GridSearchCV(sk.svm.SVR(kernel='rbf'), param_grid, n_jobs=-1)
model.fit(x_train, y_train)
print(model.best_params_)

y_pred = model.predict(x_test)
plt.plot(???, 'o', label='true')
plt.plot(???, 's', label='predicted')
plt.legend()
print("MSE =", ???, "vectors =", ??? model.best_estimator_.???)

**Вопросы:**

1. Удалось ли увеличить качество модели по сравнению с предыдущим заданием? Насколько?
1. Сравние количество опорных векторов в полученной модели и в прошлой.
1. Как изменился график предсказанной зависимости?

**Ответы:**

1. ...
1. ...

### Задание №6.2

Постройте графики зависимостей числа опорных векторов от $C$ и от $\varepsilon$.
В качестве свободных параметров используйте те, которые нашли на предыдущем шаге.

In [ ]:
%%time

_, (ax0, ax1) = plt.subplots(1, 2, figsize=(2*6, 4))

n_support = []
for C in param_grid['C']:
    m = sk.svm.SVR(???, epsilon=model.best_params_['epsilon'])
    ???

ax0.plot(param_grid['C'], n_support)
ax0.set(xlabel='C', ylabel='Количество', title='Количество векторов от C')

n_support = []
for eps in param_grid['epsilon']:
    ???

???

**Вопросы:**

1. Как влияют параметры регуляризации $C$ и $\varepsilon$ на сложность модели?

**Ответы:**

1. ...